In [60]:
import pandas as pd
import numpy as np
from haversine import haversine, Unit
import xgboost
from tabpfn.scripts import transformer_prediction_interface
from sklearn import preprocessing
from sklearn import pipeline
from sklearn import model_selection
from sklearn import linear_model
from sklearn import tree
from sklearn import svm
from sklearn import naive_bayes
from sklearn import neighbors
from sklearn import neural_network
from sklearn import ensemble
from sklearn import metrics

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(context='paper', font_scale=1.15)

data_dir = '../data/datathon_SC_ACN_22/'

### Data overview

In [61]:
orders_data = pd.read_csv(data_dir + 'orders.csv', delimiter=';', index_col='order_id')
cities_data = pd.read_csv(data_dir + 'cities_data.csv', delimiter=';')
product_data = pd.read_csv(data_dir + 'product_attributes.csv', delimiter=',', index_col='product_id')

##### City coordinate extraction

In [62]:
cities_data1 = cities_data[['city_from_name', 'city_from_coord']]
cities_data2 = cities_data[['city_to_name', 'city_to_coord']]
cities_data1 = cities_data1.rename(columns={'city_from_name': 'city_name', 'city_from_coord': 'city_coord'})
cities_data2 = cities_data2.rename(columns={'city_to_name': 'city_name', 'city_to_coord': 'city_coord'})

cities_data = pd.concat([cities_data1, cities_data2]).drop_duplicates().set_index('city_name')

# Transform lat long tuple as string to two columns
cities_data['lat'] = cities_data['city_coord'].apply(lambda x: float(x[1:-1].split(',')[0]))
cities_data['long'] = cities_data['city_coord'].apply(lambda x: float(x[1:-1].split(',')[1]))
cities_data = cities_data.drop(columns=['city_coord'])

### Feature engineering

- Lat long for all three locations
- Distance on water
- Distance on land
- Distance on land **log**
- Distance on land **sqr**
- Units
- Units **log**
- Units **sqr**
- Third party **ONH**
- Customs **ONH**
- Material handling **ONH**
- Weight
- Weight **log**
- Weight **sqr**
- Weight percent deviation from mean

In [63]:
# Replace ATHENAS with Athens and BCN with Barcelona in the origin_port column
orders_data['origin_port'] = orders_data['origin_port'].replace('ATHENAS', 'Athens')
orders_data['origin_port'] = orders_data['origin_port'].replace('BCN', 'Barcelona')

##### Order coordinate mapping

In [64]:
# Filter only for those extracted columns
city_lat_longs = cities_data[['lat', 'long']].drop_duplicates()

# Add to order df
orders_data = orders_data.join(city_lat_longs.rename(columns={'lat': 'start_lat', 'long': 'start_long'}), on='origin_port')
orders_data = orders_data.join(city_lat_longs.rename(columns={'lat': 'hub_lat', 'long': 'hub_long'}), on='logistic_hub')
orders_data = orders_data.join(city_lat_longs.rename(columns={'lat': 'end_lat', 'long': 'end_long'}), on='customer')

orders_data.loc[orders_data['logistic_hub'].isna(), 'hub_lat'] = orders_data.loc[orders_data['logistic_hub'].isna(), 'start_lat']
orders_data.loc[orders_data['logistic_hub'].isna(), 'hub_long'] = orders_data.loc[orders_data['logistic_hub'].isna(), 'start_long']

##### Shippment distance calculations

In [65]:
# Calculate distances
orders_data['distance_on_water'] = orders_data.apply(lambda x: haversine((x['start_lat'], x['start_long']), (x['hub_lat'], x['hub_long'])), axis=1)
orders_data['distance_on_land'] = orders_data.apply(lambda x: haversine((x['hub_lat'], x['hub_long']), (x['end_lat'], x['end_long'])), axis=1)

# Handle no hub orders
# if hub == nan then replace sea_distance with 0
orders_data['distance_on_water'] = orders_data['distance_on_water'].fillna(0)
orders_data.loc[orders_data['logistic_hub'].isna(), 'distance_on_land'] = orders_data.loc[orders_data['logistic_hub'].isna()].apply(lambda x: haversine((x['start_lat'], x['start_long']), (x['end_lat'], x['end_long'])), axis=1)

# Log version of distance on land
orders_data['distance_on_land_log'] = orders_data['distance_on_land'].apply(lambda x: np.log(x))

# No distance on land err handling
orders_data.loc[orders_data['distance_on_land'] == 0, 'distance_on_land_log'] = 0

# Add distance on land squared
orders_data['distance_on_land_squared'] = orders_data['distance_on_land'] ** 2

##### Order product volume

In [66]:
# Log version of units of order
orders_data['units_log'] = orders_data['units'].apply(lambda x: np.log(x))

# Units squared
orders_data['units_squared'] = orders_data['units'] ** 2

##### Product weight information and material handling

In [67]:
# Join product data
orders_data = orders_data.join(product_data, on='product_id')

# Log version of product weight
orders_data['weight_log'] = orders_data['weight'].apply(lambda x: np.log(x))

# Add squared version of weight and units
orders_data['weight_squared'] = orders_data['weight'] ** 2

# Relative deviation from mean weight
orders_data['weight_deviation'] = (orders_data['weight'] - orders_data['weight'].mean()).abs()

# Convert material handling class as int to string class
orders_data['material_handling'] = orders_data['material_handling'].apply(lambda x: "c" + str(x))

##### Drop incomplete samples

In [68]:
# Drop nas, which in this case should only be products without information
orders_data = orders_data.dropna()

##### One-hot-encode categorical attributes

In [69]:
orders_data = orders_data.sample(n=1024)

orders_features = orders_data.drop(columns=['origin_port', 'logistic_hub', 'customer', 'late_order', 'product_id'])

orders_features = pd.get_dummies(orders_features)
orders_target = orders_data['late_order']

In [70]:
orders_features.columns

Index(['units', 'start_lat', 'start_long', 'hub_lat', 'hub_long', 'end_lat',
       'end_long', 'distance_on_water', 'distance_on_land',
       'distance_on_land_log', 'distance_on_land_squared', 'units_log',
       'units_squared', 'weight', 'weight_log', 'weight_squared',
       'weight_deviation', '3pl_v_001', '3pl_v_002', '3pl_v_003', '3pl_v_004',
       'customs_procedures_CRF', 'customs_procedures_DTD',
       'customs_procedures_DTP', 'material_handling_c0.0',
       'material_handling_c1.0', 'material_handling_c2.0',
       'material_handling_c3.0', 'material_handling_c4.0',
       'material_handling_c5.0'],
      dtype='object')

#### Basic test model

In [71]:
orders_target_int = orders_target * 1

##### Training - test split

In [72]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(orders_features, orders_target_int, test_size=0.2, random_state=0)

### Model

In [77]:
model = transformer_prediction_interface.TabPFNClassifier(device='cpu', N_ensemble_configurations=64)
model.fit(X_train, y_train, overwrite_warning=True)
model.score(X_test, y_test)

Using style prior: True
Using cpu:0 device
Using a Transformer with 25.82 M parameters


0.8048780487804879

In [78]:
tn, fp, fn, tp = metrics.confusion_matrix(y_test, model.predict(X_test)).ravel()
tn, tp, fn, fp

(154, 11, 30, 10)